Note: This is (currently) the best code for identifying and analyzing clusters.

# A Possible Code for the Clusters

This code goes through the buffers one by one, rather than deconstructing all the data into one large array, like clusters_2.0 does.

First, we import Acqu (which parses the events?) and Timepix (which decodes the data??), as well as numpy, ROOT, and plotting classes of ROOT which allow us to organize and graph the data.

In [9]:
import Acqu as aq
import Timepix
import numpy as np
import ROOT
from rootpy.plotting import Hist, Hist2D, histogram, Canvas

Next, we import the data that was collected and open that data file.

In [10]:
inFile = '/w/work0/mainz/2019_05_Timepix3-Acqu/Timepix_33.dat'
aq.openFile(inFile)

Mk2 Data


Getting the data from detectors A and B:

In [36]:
TimepixAData = []
TimepixBData = []

TpxEvent = 0

def plotTimepix():
    global TpxEvent
    if(aq.epicsEvent==1):   # if there's some data (?)
        if(aq.getEpicsPV('PPOL:TIMEPIXA:EVENT')!=TpxEvent):             # if this isn't a repeat of a previous buffer
            nHitsA       = aq.getEpicsPV('PPOL:TIMEPIXA:NHITS') 
            encodedA     = aq.getEpicsPV('PPOL:TIMEPIXA:ENCODED')
            nHitsB       = aq.getEpicsPV('PPOL:TIMEPIXB:NHITS')
            encodedB     = aq.getEpicsPV('PPOL:TIMEPIXB:ENCODED')
            TimepixAData.append(Timepix.Decode(nHitsA,encodedA))        # get data from timepix A for this buffer
            TimepixBData.append(Timepix.Decode(nHitsB,encodedB))        # get data from timepix B for this buffer
            TpxEvent = aq.getEpicsPV('PPOL:TIMEPIXA:EVENT')             # note the buffer we've gotten data from already

aq.runFunction(plotTimepix,0,50000)

A function that will help us sort the data by ToA:

In [40]:
def sortThird(val):     # a function to sort arrays of data by the third entry (in our case, ToA)
    return val[2]

A function that processes the clusters in detector A or detector B:

In [27]:
def process_cluster(let):
    start_index = cl[0]                     # start index of the first hit in the cluster
    min_time = buff[start_index]['ToA']     # time of the first hit in the cluster
    meanx = 0.0                             # mean x-position of the cluster
    meany = 0.0                             # mean y-position of the cluster 
    
    n = start_index
    while(n<start_index+cn):                # go through each hit in the cluster
        if(buff[n]['ToA'] < min_time):      # if we find time smaller than the current minimum
            min_time = buff[n]['ToA']       # then reassign the smaller time to be the min time
        meanx+=buff[n]['x']                 # add up all the x-positions
        meany+=buff[n]['y']                 # add up all the y-positions
        n+=1                                # move to the next hit in the cluster
        
    meanx/=cn         # calculate the mean x-position
    meany/=cn         # calculate the mean y-position
    
    # for detector A
    if(let=='a'):
        tdposA.Fill(meanx,meany)              # plot the mean y- vs. x-position of the cluster
        multA.Fill(cn)                      # plot the number of hits in the cluster

        n = start_index
        while(n<start_index+cn):                      # for each hit in the cluster
            dtime = buff[n]['ToA'] - min_time         # find the time difference between a hit in the cluster and the min time of the cluster
            dthistA.Fill(dtime)                       # plot the time difference
            dtime_toaA.Fill(dtime,buff[n]['ToT'])     # plot ToT vs. time difference
            n+=1                                      # move on to the next hit in the cluster
    
    # for detector B
    elif(let=='b'):
        tdposB.Fill(meanx,meany)              # plot the mean y- vs. x-position of the cluster
        multB.Fill(cn)                      # plot the number of hits in the cluster

        n = start_index
        while(n<start_index+cn):                      # for each hit in the cluster
            dtime = buff[n]['ToA'] - min_time         # find the time difference between a hit in the cluster and the min time of the cluster
            dthistB.Fill(dtime)                       # plot the time difference
            dtime_toaB.Fill(dtime,buff[n]['ToT'])     # plot ToT vs. time difference
            n+=1                                      # move on to the next hit in the cluster

A function for finding clusters in detector A or detector B:

In [46]:
def findclusters(let):
    
    global buff, cl, cn, blen, b, nc
    cl = [None]*100         # a cluster can hold up to 100 hits
    cn = 0                  # counts number of hits in the current cluster
    blen = 0                # the length of the current buffer
    b = 0                   # the running/current buffer position
    nc = 0                  # count the number of clusters found
    data = []  
    
    if(let=='a'):
        data = TimepixAData    # get the data for detector A
    elif(let=='b'):
        data = TimepixBData    # get the data for detector B
    
    for i in range(len(data)):           # for each buffer in the data
        b=0                              # reset to the start of the buffer
        buff = []                        # start with a fresh buffer
        for k in range(len(data[i])):    # for each hit in the buffer
            buff.append(data[i][k])      # put that hit in our current buffer
        buff.sort(key=sortThird)         # sort our current buffer by ToA
        blen = len(buff)                 # set blen to the length of our current buffer (i.e. the number of hits)
        
        while(b<blen):           # for every position in the buffer
            cn = 0               # reset the counter for hits in the cluster (we're starting a new cluster)
            cl = [None]*100      # reset the array of cluster hits (we're starting a new cluster)
            cl[cn] = b           # assume the current position has the first hit in our current cluster  
            cn+=1                # increment the number of hits for this cluster

            c=b+1       # start comparing at position c that is one above the current buffer position
            while((c<b+99) and (c<blen)):                         # check all positions up to 99 away, so long as we're within the buffer
                if(abs(buff[c]['ToA']- buff[b]['ToA'] < 100)):    # if a hit c is within 100ns of the current buffer position
                    cl[cn] = c                                    # save this position to the cluster
                    cn+=1                                         # increment the number of hits for this cluster
                else:
                    process_cluster(let)       # analyze the cluster we've found
                    b+=cn                      # move beyond this cluster
                    cn = 0                     # reset the counter for hits in the cluster (we're starting a new cluster)
                    cl = [None]*100            # reset the array of cluster hits
                    nc+=1                      # increment the number of clusters we found

                    if(c<b+99 and c<blen):     # if we're staring in the while loop after processing this cluster
                        cl[cn] = b             # assume the current position has the first hit in our new cluster
                        cn+=1                  # increment the number of hits for this cluster  
                        
                c+=1     # move to the next position for comparison
                
                if(((c>=blen) and cn!=0)or((c>=b+99) and cn!=0)):  # if exiting the loop but still have a cluster to process
                    process_cluster(let)                           # process the last cluster
                    b+=cn                                          # move beyond this cluster
                    cn = 0                                         # reset the counter for hits in the cluster
                    cl = [None]*100                                # reset the array of cluster hits
                    nc+=1                                          # increment the number of clusters we found
                    
            b+=1     # move to the next buffer position
            
        

Running and graphing the data for detector A:

In [45]:
ROOT.enableJSVis()
c1 = ROOT.TCanvas("c1","Clusters by Buffers", 1000, 2000)
c1.Divide(2,4)

# for detector A

tdposA = Hist2D(256,0,256,256,0,256)
tdposA.GetXaxis().SetTitle("mean x position of the cluster")
tdposA.GetYaxis().SetTitle("mean y position of the cluster")

multA = Hist(50,0,50)
multA.GetXaxis().SetTitle("number of hits in the cluster")
multA.GetYaxis().SetTitle("number of clusters")

dthistA = Hist(100,0,100)
dthistA.GetXaxis().SetTitle("time diff between a hit and the min time of the cluster")
dthistA.GetYaxis().SetTitle("number of clusters")

dtime_toaA = Hist2D(100,0,100,230,0,230)
dtime_toaA.GetXaxis().SetTitle("time diff between a hit and the min time of the cluster")
dtime_toaA.GetYaxis().SetTitle("ToT")

findclusters('a')

c1.cd(1)
tdposA.Draw("colz")
c1.cd(2)
multA.Draw()
c1.cd(3)
dthistA.Draw()
c1.cd(4)
dtime_toaA.Draw("colz")

# for detector B

tdposB = Hist2D(256,0,256,256,0,256)
tdposB.GetXaxis().SetTitle("mean x position of the cluster")
tdposB.GetYaxis().SetTitle("mean y position of the cluster")

multB = Hist(50,0,50)
multB.GetXaxis().SetTitle("number of hits in the cluster")
multB.GetYaxis().SetTitle("number of clusters")

dthistB = Hist(100,0,100)
dthistB.GetXaxis().SetTitle("time diff between a hit and the min time of the cluster")
dthistB.GetYaxis().SetTitle("number of clusters")

dtime_toaB = Hist2D(100,0,100,230,0,230)
dtime_toaB.GetXaxis().SetTitle("time diff between a hit and the min time of the cluster")
dtime_toaB.GetYaxis().SetTitle("ToT")

findclusters('b')

c1.cd(5)
tdposB.Draw("colz")
c1.cd(6)
multB.Draw()
c1.cd(7)
dthistB.Draw()
c1.cd(8)
dtime_toaB.Draw("colz")

c1.Draw()

[(14, 25, 12700, 15, 49), (171, 114, 37385, 20, 45), (171, 115, 37386, 26, 45), (153, 105, 39185, 15, 105), (153, 106, 39186, 11, 47), (152, 105, 39187, 10, 7), (93, 113, 39212, 5, 67), (92, 113, 39213, 2, 32), (148, 230, 61859, 7, 101), (181, 167, 68548, 16, 119), (99, 98, 72219, 18, 122), (98, 98, 72221, 29, 61), (230, 1, 126179, 26, 13), (230, 0, 126179, 23, 21), (231, 0, 126180, 23, 27), (192, 184, 137724, 12, 119), (252, 178, 150447, 12, 33), (251, 178, 150448, 15, 55), (167, 72, 151195, 21, 123), (203, 132, 164556, 14, 144), (220, 126, 164557, 14, 4), (219, 126, 164557, 12, 23), (217, 125, 164557, 9, 60), (215, 125, 164557, 5, 18), (218, 125, 164557, 13, 49), (214, 125, 164559, 16, 3), (185, 175, 188150, 4, 114), (152, 90, 192857, 10, 46), (153, 90, 192857, 16, 94), (152, 91, 192859, 18, 34), (118, 157, 234432, 20, 51), (118, 158, 234433, 17, 38), (120, 130, 318033, 8, 100), (119, 130, 318035, 5, 40), (143, 170, 320612, 12, 86), (156, 122, 343752, 10, 42), (156, 121, 343753, 13, 

[(129, 98, 5485, 12, 102), (133, 122, 7216, 24, 33), (133, 123, 7216, 17, 61), (135, 124, 7217, 15, 15), (134, 123, 7217, 27, 55), (136, 124, 7217, 18, 75), (132, 123, 7217, 21, 28), (132, 122, 7218, 28, 8), (9, 219, 22197, 18, 15), (180, 220, 52341, 15, 100), (176, 173, 92974, 9, 77), (110, 153, 97093, 19, 136), (219, 20, 98513, 10, 40), (218, 20, 98513, 4, 64), (102, 27, 121591, 16, 125), (241, 175, 156267, 20, 86), (241, 176, 156268, 12, 22), (166, 216, 163962, 15, 87), (166, 217, 163964, 21, 22), (169, 123, 169729, 16, 108), (145, 142, 204622, 15, 57), (144, 142, 204623, 23, 56), (198, 124, 215071, 22, 80), (197, 124, 215073, 27, 20), (160, 221, 217002, 3, 54), (160, 222, 217003, 6, 35), (159, 221, 217004, 7, 12), (159, 222, 217004, 8, 22), (160, 0, 219055, 0, 55), (159, 0, 219056, 10, 29), (159, 203, 230676, 10, 10), (156, 201, 230676, 14, 23), (161, 205, 230676, 9, 8), (162, 204, 230676, 20, 29), (163, 202, 230676, 20, 38), (155, 201, 230676, 17, 163), (163, 201, 230676, 20, 62),

[(187, 231, 38240, 13, 36), (155, 172, 44598, 14, 46), (168, 218, 44716, 21, 25), (112, 223, 71290, 11, 31), (109, 228, 71290, 12, 25), (109, 229, 71290, 12, 38), (109, 227, 71291, 20, 6), (111, 224, 71291, 15, 4), (112, 224, 71291, 21, 13), (110, 225, 71291, 23, 28), (109, 226, 71291, 21, 12), (169, 133, 146579, 25, 70), (208, 120, 150678, 17, 42), (124, 222, 151011, 7, 70), (123, 222, 151012, 13, 11), (161, 254, 180941, 12, 27), (161, 253, 180941, 13, 36), (249, 57, 204756, 6, 60), (136, 85, 207742, 18, 32), (153, 218, 234556, 17, 36), (141, 175, 317098, 17, 45), (242, 18, 357034, 11, 7), (242, 19, 357034, 10, 16), (146, 181, 366061, 18, 22), (146, 182, 366061, 17, 41), (108, 181, 409237, 17, 35), (114, 221, 442205, 8, 8), (115, 221, 442205, 15, 17), (182, 39, 445200, 7, 25), (112, 181, 465116, 20, 21), (117, 210, 493043, 9, 31), (117, 211, 493044, 16, 11), (117, 179, 494384, 7, 21), (117, 178, 494385, 13, 4), (149, 251, 495019, 9, 6), (171, 25, 496603, 20, 20), (254, 123, 514876, 3,

[(192, 71, 26175, 6, 11), (192, 70, 26175, 13, 24), (22, 145, 35095, 15, 63), (238, 168, 35257, 15, 13), (238, 169, 35257, 15, 27), (172, 139, 96859, 11, 22), (173, 139, 96859, 12, 54), (148, 19, 131587, 20, 48), (187, 211, 135047, 8, 53), (127, 230, 147166, 15, 15), (134, 156, 189218, 25, 64), (149, 113, 208588, 7, 32), (211, 186, 208791, 7, 49), (125, 252, 222194, 13, 24), (125, 253, 222194, 12, 42), (128, 254, 247048, 4, 6), (128, 253, 247048, 6, 30), (185, 70, 271716, 8, 22), (185, 71, 271717, 13, 3), (96, 126, 273721, 13, 39), (86, 5, 297460, 10, 35), (171, 39, 302679, 20, 75), (192, 54, 305974, 12, 3), (193, 54, 322357, 11, 8), (103, 2, 326300, 18, 46), (112, 195, 371237, 9, 25), (111, 195, 371237, 9, 45), (211, 80, 399088, 8, 96), (211, 81, 399089, 22, 16), (171, 32, 419843, 12, 38), (174, 38, 419844, 24, 37), (142, 140, 455831, 18, 24), (110, 250, 476164, 15, 75), (145, 204, 500721, 15, 16), (146, 203, 500721, 20, 42), (145, 203, 500721, 21, 69), (176, 46, 507229, 21, 31), (74,